In [ ]:
import os 
import torch
import cv2
import matplotlib.pyplot as plt
import glob
from torch.utils.data import DataLoader

Put bbox info in list of lists
make a label class id list 
make them tensor
https://pytorch.org/docs/stable/data.html

# Record the meeting

# Record the meeting


# Record the meeting


# Record the meeting

# Record the meeting

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
with zipfile.ZipFile("/content/drive/MyDrive/PSHS_Projects/crop_and_weed.zip","r") as zip_ref:
    zip_ref.extractall()

# Structure the data so that it can fed to the neural network

# How to do we extract bboxes from files

In [ ]:
images = sorted(glob.glob("/content/agri_data/data/*.jpeg"))
text = sorted(glob.glob("/content/agri_data/data/*.txt"))
classes = ['crop', 'weed'] # Crop is labeled 0 

def yolo2voc(yolo_bbox): 

  pascal_voc_bbox = [0, 0, 0, 0]
  pascal_voc_bbox[0] = (yolo_bbox[0] - yolo_bbox[2]/2) * 512
  pascal_voc_bbox[1] = (yolo_bbox[1] - yolo_bbox[3]/2) *512
  pascal_voc_bbox[2] = (yolo_bbox[0] + yolo_bbox[2]/2) *512
  pascal_voc_bbox[3] = (yolo_bbox[1] + yolo_bbox[3]/2) * 512

  return pascal_voc_bbox


class LoadAgriData(): 

  def __init__(self, full_images, full_text): 
    self.full_images = full_images
    self.full_text = full_text
    
  def __getitem__(self, x): 
    image_file = self.full_images[x] 
    text_file = self.full_text[x]
    image_id = x 

    img = cv2.imread(image_file)
    print(img.shape)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    boxes = []
    labels = []

    # Extract numbers from text file 
    with open(text_file) as f:
      lines = f.readlines()
      for bbox in lines: 
        lines_split = bbox.split()
        lines_split = [float(value) for value in lines_split]
        yolo_voc = lines_split[1:]
        label = lines_split[0]
        pascal_voc_bboxes = yolo2voc(yolo_voc)
        boxes.append(pascal_voc_bboxes)
        labels.append(label)
    

    target = {}
    target["boxes"] = boxes
    target["labels"] = labels
    target["image_id"] = image_id

    return img, target

  def __len__(self): 
    return len(self.full_images)




  # How do you get one at once

batch_size = 12 # or 8 is the Goldilocks or perfect batch size
agri_data = LoadAgriData(images, text)
agri_dataloader = DataLoader(agri_data, batch_size = batch_size, shuffle = True)

agri_iterator = iter(agri_dataloader)
data = next(agri_iterator)

(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)


In [ ]:
data[1]

{'boxes': [[tensor([  0.9999, 213.9999, 146.0001,  48.9999,  26.9998,   0.9997,  36.0000,
            25.9999, 132.0003,  61.9999,  61.0002,  32.0000], dtype=torch.float64),
   tensor([ 16.9999, 201.9999, 213.0002, 155.0001,  79.0001,  52.9999, 104.0000,
            64.0003, 164.9999,   9.0002, 139.9997,  40.9999], dtype=torch.float64),
   tensor([511.0001, 324.9999, 431.0003, 475.9997, 428.9997, 509.9999, 499.0001,
           386.9998, 408.0000, 475.0001, 408.0000, 400.0000], dtype=torch.float64),
   tensor([502.0001, 313.9999, 414.0001, 403.0001, 464.9999, 430.9998, 397.0002,
           410.0001, 398.9998, 481.0002, 440.0000, 382.0001], dtype=torch.float64)]],
 'image_id': tensor([  46,  768,  983,   27, 1070, 1245,  860,  611,  222, 1097,  406,   63]),
 'labels': [tensor([0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0.], dtype=torch.float64)]}

In [ ]:
def generator(a):
  for num in a: 
    yield num + 10

a = [1, 2, 3, 4, 5]
our_generator = generator(a)


In [ ]:
our_generator

<generator object generator at 0x7f9861463cd0>

In [ ]:
iter_generator = iter(our_generator)


In [ ]:
next(iter_generator)

StopIteration: ignored

In [ ]:
agri_data = LoadAgriData(images, text)

len(agri_data)

1300

# Delete Hard coded thing

In [ ]:
building = BluePrint()

building[8]

In [ ]:
entire_data = [1, 2, 3, 4, 5, 6, 7, 8]

entire_data[0]

1

In [ ]:
class BluePrint(): 
  def __init__(self, size, color, durability, location = "Phillipines"): 
    self.color = color
    self.size = size
    self.location = location
  
  def rebuild(self): 
    print("I am going to rebuild")

  def shutdown(self, time): 
    print("Shutdown for time: {}".format(time))

  def __getitem__(self, x): 
    print(x)


In [ ]:
building = BluePrint(size = 10, color = "RED", durability = "hard")

In [ ]:
building[]

10


In [ ]:
building.location

'Phillipines'